### Tools

#### Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

- A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
- A function or coroutine to execute.

In [21]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

model = init_chat_model('gpt-4')
response = model.invoke("Why do parrots talk?")
response

AIMessage(content="Parrots talk as a form of communication, although they don't understand the language the way humans do. They mimic or imitate sounds in their environment, which often include human speech if they live in a home with people. Parrots are social animals and talking helps them bond with their human caregivers. It’s also a form of enrichment and mental stimulation for them.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 13, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D4cZtMSr5bspNF8WiBcmmit3ibSmP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c1bdc-beca-77b3-82e6-e06e294bda47-0', tool_ca

In [22]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # Dummy implementation for illustration
    return f"The current weather in {location} is sunny with a temperature of 25°C."

model_with_tools = model.bind_tools([get_weather])

In [23]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 56, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D4ca3GwhBNEutb4uJd8wnUN0dAAET', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c1bdc-e4b4-7240-843c-39718913c5f3-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_AP63n9x9q4K283myYhGbmQMH', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 56, 'output_tokens': 15, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Tool: get_weather
Args: {'location': 'Boston'}


### Tool Execution Loops

In [24]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The current weather in Boston is sunny with a temperature of 25°C.


In [25]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 55, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D4caFmYtO3vnTZFEVBqoG0hxQh181', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c1bdd-1524-70c1-b2b1-c7cbcf053271-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_F6A1ACXfK68hcpuDtlW9p0Ok', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 55, 'output_tokens': 15, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': 